<a href="https://colab.research.google.com/github/JhonatanPeixoto/JhonatanPeixoto/blob/main/Projeto_Final(Allan%2C_Fabio%2C_Jhonatan%2C_Vitor)_Estudo_de_Caso_TripAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
!pip install selenium

!apt-get update
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, 
            '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.

In [312]:
# Importar biblioteca de regex
import re

# Importar biblioteca nltk para remoção de stop words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Biblioteca para remoção de caracteres especiais
from unicodedata import normalize
from string import punctuation

In [313]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [314]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [315]:
browser = webdriver.Chrome('chromedriver', options=chrome_options)

In [316]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303324-Goiania_State_of_Goias.html'

In [317]:
browser.get(url)

In [318]:
b = bs(browser.page_source)

In [319]:
rests = b.find('div', attrs={'data-test-target':'restaurants-list'})

In [320]:
class_alvo = rests.find('div').find('div')['class'][0]

In [321]:
df = pd.DataFrame(columns=['nome', 'avaliacao', 'descricao'])

In [322]:
linha = 0
for rest in rests.find_all('div', attrs={'class':class_alvo}):
    titulo = rest.find_all('a')[1].text
    avaliacao  = rest.find_all('a')[2].text
    desc   = rest.find_all('span')[14].text
    
    if re.search('\$', desc):
        desc   = rest.find_all('span')[12].text

    df.loc[linha] = [titulo, avaliacao, desc]
    linha+=1

###tratar coluna nome:remover numeração com regex, 
###tratar coluna avaliação: remover o nome "avaliações", converter para int
###tratar coluna descrição: caixa baixa, remover preposições, acentuações, etc


In [323]:
stop_words = stopwords.words('portuguese')

In [324]:
def remover_sw(texto):
    words_token = word_tokenize(texto)
    
    # sentenca_limpa = []
    # for w in words_token:
    #     if not w.lower() in stop_words:
    #         sentenca_limpa.append(w)

    sentenca_limpa = [w for w in words_token if not w.lower() in stop_words]

    return ' '.join(sentenca_limpa)

In [325]:
def limpar_texto(texto):
    # Tratando caracteres especiais
    texto = normalize('NFKD', texto). encode('ASCII', 'ignore').decode('ASCII')
    # Removendo pontuações
    texto = re.sub(f'[{punctuation}]', '', texto)
    # Removendo número
    texto = re.sub('[0-9]', '', texto)
    # Colocar em caixa baixa
    texto = texto.lower()
    # Removendo stop words
    texto = remover_sw(texto)

    return texto

In [326]:
%%time
df['descricao'] = df['descricao'].map(limpar_texto)

CPU times: user 6.64 ms, sys: 0 ns, total: 6.64 ms
Wall time: 9.39 ms


In [346]:
df['nome'] = df['nome'].map(lambda n:n.split(".")[1])

IndexError: ignored

In [347]:
df

,nome,avaliacao,descricao
0,Coco Bambu Flamboyant,4986,brasileira frutos mar
1,Taji Japanese Cuisine,831,japonesa asiatica
2,El Argentino,334,steakhouse argentina
3,Íz Restaurante,872,francesa brasileira
4,Restaurante Popular,311,brasileira
5,Carne de Sol 1008,2341,steakhouse brasileira
6,Ciao Ciao Cucina,178,italiana bar
7,Madero,1140,americana steakhouse
8,Pizzaria Pitigliano,722,italiana pizza
9,Outback Steakhouse - Shopping Flamboyant,1807,steakhouse bar


In [348]:
df['avaliacao'] = df['avaliacao'].map(lambda a:a.replace(' avaliações','').replace('.',''))

In [349]:
df['avaliacao'].astype(int)

0     4986
1      831
2      334
3      872
4      311
5     2341
6      178
7     1140
8      722
9     1807
10     310
11     208
12     450
13     514
14     463
15    1004
16     441
17     420
18     338
19     189
20     402
21     354
22     537
23     307
24     279
25     776
26     993
27     346
28     111
29     204
Name: avaliacao, dtype: int64

##Aplicação do sistema de recomendação

In [350]:
df['descricao']

0           brasileira frutos mar
1               japonesa asiatica
2            steakhouse argentina
3             francesa brasileira
4                      brasileira
5           steakhouse brasileira
6                    italiana bar
7            americana steakhouse
8                  italiana pizza
9                  steakhouse bar
10                 brasileira bar
11       brasileira internacional
12                 brasileira bar
13           americana lanchonete
14                 steakhouse bar
15          portuguesa brasileira
16                     brasileira
17                 brasileira bar
18                 italiana pizza
19            europeia portuguesa
20          steakhouse brasileira
21           brasileira churrasco
22        brasileira sulamericana
23          brasileira frutos mar
24            europeia portuguesa
25                 brasileira bar
26                 brasileira bar
27          brasileira frutos mar
28    internacional contemporanea
29       brasi

In [351]:
tipo_culinaria = df['descricao'].str.split(' ').astype(str)

In [ ]:
tipo_culinaria.head(30)

In [353]:
#importação de bibliotecas para o procedimento

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [354]:
# Criando uma instância do tfidf
vectorizer = TfidfVectorizer(analyzer='word',
                             ngram_range=(1,3))

# Definir os valores da nossa matriz de características
vetores = vectorizer.fit_transform(tipo_culinaria)

In [355]:
# Criar um dataframe com as características (features) e os pesos
# associados a estas
words_df = pd.DataFrame(vetores.toarray(),
                        columns=vectorizer.get_feature_names_out())

In [356]:
cosseno_tipo_culinaria = cosine_similarity(words_df, words_df)


In [357]:
#df.shape

In [358]:
cosseno_tipo_culinaria[0][:]

array([1.        , 0.        , 0.        , 0.05787202, 0.21353273,
       0.07196533, 0.        , 0.        , 0.        , 0.        ,
       0.0843967 , 0.0608332 , 0.0843967 , 0.        , 0.        ,
       0.06298539, 0.21353273, 0.0843967 , 0.        , 0.        ,
       0.07196533, 0.05787202, 0.05787202, 1.        , 0.        ,
       0.0843967 , 0.0843967 , 1.        , 0.        , 0.0608332 ])

In [359]:
for i,v in enumerate(cosseno_tipo_culinaria[2]):
    if v > 0.4:
        print(i, v)

2 1.0


In [360]:
catalogo_restaurantes = pd.Series(df.index, index=df['nome'].str.strip())

In [361]:
catalogo_restaurantes.index

Index(['Coco Bambu Flamboyant', 'Taji Japanese Cuisine', 'El Argentino',
       'Íz Restaurante', 'Restaurante Popular', 'Carne de Sol 1008',
       'Ciao Ciao Cucina', 'Madero', 'Pizzaria Pitigliano',
       'Outback Steakhouse - Shopping Flamboyant', 'Glória Bar e Restaurante',
       'Panela Mágica', 'Celsin & Cia', 'Lifebox Burger Gourmet',
       'Madero Flamboyant Shopping', 'Botequim Mercatto', 'Chão Nativo',
       'Bahrem', 'Casa São Paulo', 'Porto Cave', 'Churrascaria Gramado',
       'Churrascaria DO Walmor', 'Kabanas Restaurante e Bar', 'Bar do Peixe',
       'Obelisque', 'Empório Saccaria', 'Empório Piquiras',
       'Tucunaré na Chapa', 'Meze Restaurante', 'Las Nenas Bistrô'],
      dtype='object', name='nome')

In [362]:
catalogo_restaurantes['Chão Nativo']

16

In [363]:
import string
def recomendar_restaurante(nome_restaurante, mat_similaridades):
    # Obtendo o índice com base no título informado
    idx = catalogo_restaurantes[nome_restaurante] # 'barriga cheia'

    # Obter similaridades para o item selecionado
    sim = (enumerate(mat_similaridades[idx]))

    # Reordenar nossas similaridades em ordem decrescente de recomendação
    sim = sorted(sim, key=lambda x:x[1], reverse=True)

    # Filtrar os top 10
    sim = sim[0:11] 

    # Obter os índices de restaurantes recomendados
    indices_recomendados = [x[0] for x in sim]
    recomendados_sim     = [x[1] for x in sim]

    return pd.DataFrame({
        'Nome do restaurante':df['nome'].iloc[indices_recomendados].values,
        'Similaridade':recomendados_sim,
        'Avaliação:':df['avaliacao'].iloc[indices_recomendados].values,
        'Descricao:':df['descricao'].iloc[indices_recomendados].values,
    })

In [ ]:
catalogo_restaurantes

In [ ]:
from pandas.core.dtypes.cast import astype_array
print('#'*5 + 'SISTEMA DE RECOMENDAÇÃO' + '#'*5)

for nome,id in catalogo_restaurantes.iteritems():
  nome = str.strip(nome)
  id = str(id).zfill(3)
  print(f'ID:{id} NOME REST: {nome}')
rest_escolhido = input('Informe o nome do restaurante desejado: ')

recomendar_restaurante(rest_escolhido, cosseno_tipo_culinaria)


